In [1]:
import sqlite3
import csv
import pandas as pd
import os
import re

from nltk import word_tokenize, sent_tokenize # widely used text tokenizer
from nltk.stem.porter import PorterStemmer # an approximate method of stemming words
stemmer = PorterStemmer()

from difflib import SequenceMatcher as SeqMatcher
import numpy as np


# def valid_page(page):
#     crawl_id, url, is_pdf, cur_depth, body = page
#     return True

# pickled_df = "../../processed_df.pkl"
folder_prefix = '/home/jovyan/work/'
folder_prefix
#conn = sqlite3.connect(folder_prefix + 'scrapy_cluster_data/data.db', timeout=30)



'/home/jovyan/work/'

In [2]:
#new_data = pd.read_pickle(folder_prefix+ 'scrapy_cluster_data/new_processed_df.pkl')
new_data = pd.read_pickle(folder_prefix+ 'charters_full_2015.pkl')

In [3]:
new_data

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,FTE,YEAR_OPENED,YEAR_CLOSED,SEARCH16,TOTETH,PCTETH,AGE,PCTFRL,PLACE,STR
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-1.00,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000
1,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-1.00,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000
2,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-1.00,2005.0,NaN,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000
3,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-1.00,2005.0,NaN,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000
4,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,-1.00,1998.0,NaN,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000
5,Arizona Agribusiness & Equine Center,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,-1.00,1998.0,NaN,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000
6,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY MIDDLE,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,40.82,2013.0,NaN,"CUMBERLAND ACADEMY MIDDLE 1040 SHILOH RD , TY...",240.0,0.471513,2.0,0.379175,City,12.469378
7,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND H S,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,NaN,2015.0,NaN,"CUMBERLAND H S 7200 PALUXY RD , TYLER TX 7570...",59.0,0.385621,0.0,NaN,Rural,NaN
8,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,36.23,1998.0,NaN,"CUMBERLAND ACADEMY 1340 SHILOH RD , TYLER TX ...",217.0,0.434870,17.0,0.408818,City,13.773116
9,ACCELERATED INTERMEDIATE ACADEMY,281.0,ACCELERATED INTERDISCIPLINARY ACAD,TX,3.0,http://www.aiacharterschools.org/,1.0,TX,NaN,NaN,...,16.85,2004.0,NaN,ACCELERATED INTERDISCIPLINARY ACAD 12825 SUMM...,276.0,0.982206,11.0,0.864769,City,16.676558


In [4]:
new_data.dtypes

CMO_NAME                     object
CMO_MEMSUM                  float64
SCH_NAME                     object
CMO_STATE                    object
CMO_SCHNUM                  float64
CMO_URL                      object
CMO_NUMSTATES               float64
CMO_ALLSTATES                object
CMO_SECTOR                   object
CMO_NUMSTUDENTS_CREDO17     float64
CMO_TYPE                     object
CMO_WEBTEXT                  object
SURVYEAR                     object
FIPST                       float64
STABR                        object
SEANAME                      object
LEAID                         int64
ST_LEAID                     object
SCHID                       float64
ST_SCHID                     object
NCESSCH                     float64
MSTREET1                     object
MSTREET2                     object
MSTREET3                     object
MCITY                        object
MSTATE                       object
MZIP                        float64
MZIP4                       

In [5]:
new_data['WEBTEXT'][600][10][3]

"Dalzell Lance Campus\nSkip to main content\nCNCA\nAbout Us\nWelcome\nAbout Dalzell Lance\nLeadership\nStaff Directory\nContact Us\nAcademics\nAcademic Calendar\nBell Schedule\nAcademic Calendar with Bell Schedule\nPrograms\nResults\nSchool Accountability Report Card\nAcademic Departments\nStudents\nStudent Resources\nEvent Calendar\nCollege Center\nPowerSchool\nAchieve3000\nKickboard\nClubs and Sports\nAlumni\nBreakfast & Lunch Menu\nParents\nEvent Calendar\nParent Resources\nPowerSchool\nAchieve3000\nKickboard\nBreakfast & Lunch Menu\nEnroll\nStaff\nPowerSchool\nStaff Email\nAchieve3000\nKickboard\nIlluminate\nGoogle Apps\nStaff Webpage\nIT Support\nInventory\nMedia\nNews and Announcements\nPhoto Album\nPress Releases\nSearch\nEnglish\nEspañol\nNews and Announcements\n\t\t\t\t\t\t\tAnnual Parent Survey is Now Available\t\t\t\t\t\t\n\t\t\t\t\t\t\tPlease take a moment to complete the Annual Parent Survey. Your feedback is very important to us!\t\t\t\t\t\t\t\n Read More » \t\t\t\t\t\t\t

In [7]:
text = new_data['data'][0][4][3]
print(re.sub(r"\t\n\s", ",", text))

Skip navigation
Select a School
DISTRICT
CCPS
Elementary
Anderson Elementary
Arnold Elementary
Brown Elementary
Callaway Elementary
Church Street Elementary
East Clayton Elementary
Edmonds Elementary
Fountain Elementary
Harper Elementary
Hawthorne Elementary
Haynie Elementary
Huie Elementary
Jackson Elementary
Kemp Elementary
Kemp Primary
Kilpatrick Elementary
King Elementary
Lake City Elementary
Lake Ridge Elementary
Lee Street Elementary
Marshall Elementary
McGarrah Elementary
Morrow Elementary
Mt. Zion Elementary
Mt. Zion Primary
Northcutt Elementary
Oliver Elementary
Pointe South Elementary
River's Edge Elementary
Riverdale Elementary
Smith Elementary
Suder Elementary
Swint Elementary
Tara Elementary
Unidos Dual Language School
West Clayton Elementary
Middle
Adamson Middle
Babb Middle
Eddie White Academy (K-8)
Forest Park Middle
Jonesboro Middle
Kendrick Middle
Lovejoy Middle
M.D. Roberts Middle
Morrow Middle
Mundy's Mill Middle
North Clayton Middle
Pointe South Middle
Rex Mill Mid

In [15]:
new_data['data'][0][4][3]

"Skip navigation\nSelect a School\nDISTRICT\nCCPS\nElementary\nAnderson\xa0Elementary\nArnold Elementary\nBrown Elementary\nCallaway Elementary\nChurch Street Elementary\nEast Clayton Elementary\nEdmonds Elementary\nFountain Elementary\nHarper Elementary\nHawthorne Elementary\nHaynie Elementary\nHuie Elementary\nJackson Elementary\nKemp Elementary\nKemp Primary\nKilpatrick Elementary\nKing Elementary\nLake City Elementary\nLake Ridge Elementary\nLee Street Elementary\nMarshall Elementary\nMcGarrah Elementary\nMorrow Elementary\nMt. Zion Elementary\nMt. Zion Primary\nNorthcutt Elementary\nOliver Elementary\nPointe South Elementary\nRiver's Edge Elementary\nRiverdale Elementary\nSmith Elementary\nSuder Elementary\nSwint Elementary\nTara Elementary\nUnidos Dual Language School\nWest Clayton Elementary\nMiddle\nAdamson Middle\nBabb Middle\nEddie White Academy (K-8)\nForest Park Middle\nJonesboro Middle\nKendrick Middle\nLovejoy Middle\nM.D. Roberts Middle\nMorrow Middle\nMundy's Mill Middl

In [6]:
#Create overlap parsing method

keywords = ['values', 'academics', 'skills', 'purpose',
                       'direction', 'mission', 'vision', 'vision', 'mission', 'our purpose',
                       'our ideals', 'ideals', 'our cause', 'curriculum','curricular',
                       'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system',
                       'structure','philosophy', 'philosophical', 'beliefs', 'believe',
                       'principles', 'creed', 'credo', 'values','moral', 'history', 'our story',
                       'the story', 'school story', 'background', 'founding', 'founded',
                       'established','establishment', 'our school began', 'we began',
                       'doors opened', 'school opened', 'about us', 'our school', 'who we are',
                       'our identity', 'profile', 'highlights']

mission_keywords = ['mission','vision', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective']
curriculum_keywords = ['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure']
philosophy_keywords = ['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value',  'moral']
history_keywords = ['history', 'story','our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began', 'we began', 'doors opened', 'school opened']
about_keywords =  ['about us', 'our school', 'who we are', 'overview', 'general information', 'our identity', 'profile', 'highlights']

mission_keywords = set(stemmer.stem(word) for word in mission_keywords)
curriculum_keywords = set(stemmer.stem(word) for word in curriculum_keywords)
philosophy_keywords = set(stemmer.stem(word) for word in philosophy_keywords)
history_keywords = set(stemmer.stem(word) for word in history_keywords)
about_keywords =  set(stemmer.stem(word) for word in about_keywords)


In [97]:
c = new_data.iloc[10]['WEBTEXT']
c

[('http://www.aiacharterschools.org/',
  'False',
  '0',
  "Skip to Main Content\nToggle main menu visibility \nHome\nAbout Us\nAbout Our School\nOur History\nAdministration\nBoard Members\nSchools\nHouston Elementary\nHouston Middle School\nLancaster Campus\nRegistration\nCalendar\nNews\nDepartments\nPreschool\nFood Services\nHuman Resources\nHelpful Links\nContact Us\nACCELERATED INTERDISCIPLINARY INTERMEDIATE ACADEMY\n“The school where every challenge becomes a mission accomplished.\n”\nWelcome to Our District's Website\nThank you for visiting the AIA District website! We hope you are impressed by what we have to offer. Those in our district enjoy a friendly atmosphere, and we invite you to take part in our community.\nThe AIA District is an open enrollment school, serving students in Houston and East Texas. To learn more about our district, take some time to browse through our website and see what we have to offer our students, parents, and community. You’ll find links to our \nsch

In [63]:
#all_pages is a list of strings of all web page texts
all_pages = []

#all reverse pages reverses each page, and that's ok, cuz if you have the same words reversed in a lot of strings 
#it's still a "overlapping" phrase, though its garbled guck to us
all_reverse_pages = []
col_pages = new_data.iloc[17:18]['WEBTEXT'] #change to new_data['data'] later, but work with first 1 schools for now
i = 0
count = 0
for school_data in col_pages:
    print ("School : " + str(i) +"\n")
    for tup in school_data:
        print(tup[0])
        count+=1
        all_pages.append(tup[3])
        all_reverse_pages.append(tup[3][::-1])
    i+=1

#all_pages

School : 0

https://www.aristoiclassical.org/
https://www.aristoiclassical.org/apps/bell_schedules/
https://www.aristoiclassical.org/apps/contact/
https://www.aristoiclassical.org/apps/events/
https://www.aristoiclassical.org/apps/events/2018/6/11/3859790/?REC_ID=3859790&d=2018-6-11&id=0
https://www.aristoiclassical.org/apps/events/2018/6/18/3859790/?REC_ID=3859790&d=2018-6-18&id=0
https://www.aristoiclassical.org/apps/events/2018/6/25/3859790/?REC_ID=3859790&d=2018-6-25&id=0
https://www.aristoiclassical.org/apps/events/2018/7/16/4942873/?REC_ID=4942873&d=2018-7-16&id=0
https://www.aristoiclassical.org/apps/events/2018/7/17/4942873/?REC_ID=4942873&d=2018-7-17&id=0
https://www.aristoiclassical.org/apps/events/2018/7/18/4942873/?REC_ID=4942873&d=2018-7-18&id=0
https://www.aristoiclassical.org/apps/news/
https://www.aristoiclassical.org/apps/news/article/844826
https://www.aristoiclassical.org/apps/news/article/849214
https://www.aristoiclassical.org/apps/news/article/860301
https://www.a

In [64]:
new_data.head(7)['WEBTEXT']

0    [(https://www.aaechighschools.com/public-chart...
1    [(https://www.aaechighschools.com/public-chart...
2    [(https://www.aaechighschools.com/public-chart...
3    [(https://www.aaechighschools.com/public-chart...
4    [(https://www.aaechighschools.com/index.php, F...
5    [(https://www.aaechighschools.com/index.php, F...
6    [(https://ms.cumberlandacademy.com/, False, 0,...
Name: WEBTEXT, dtype: object

col_pages

In [65]:
print ("number of all web page texts: " + str(len(all_pages)))
#all_pages is a list of strings of all web page texts

print ("number of all reversed web page texts: " + str(len(all_reverse_pages)))
count

number of all web page texts: 219
number of all reversed web page texts: 219


219

In [33]:
print(all_pages[1])

Skip to Main Content
Toggle main menu visibility 
Home
About Us
About Our School
Our History
Administration
Board Members
Schools
Houston Elementary
Houston Middle School
Lancaster Campus
Registration
Calendar
News
Departments
Preschool
Food Services
Human Resources
Helpful Links
Contact Us
ACCELERATED INTERDISCIPLINARY INTERMEDIATE ACADEMY
“The school where every challenge becomes a mission accomplished.
”
Welcome to Our District's Website
Thank you for visiting the AIA District website! We hope you are impressed by what we have to offer. Those in our district enjoy a friendly atmosphere, and we invite you to take part in our community.
The AIA District is an open enrollment school, serving students in Houston and East Texas. To learn more about our district, take some time to browse through our website and see what we have to offer our students, parents, and community. You’ll find links to our 
schools
, information about 
enrollment
, and 
employment opportunities
.
Quick Links
Pare

In [34]:
print(all_reverse_pages[0])


sretsambeW loohcS
 yb deniatniam dna dengised etiS | 8102© thgirypoC						
8926.382 )317( leT							
rebmuN eciffO
	64157 XT ,retsacnaL
.dR eniltleB tsaE 109
retsacnaL AIA
       58077 XT ,notsuoH
.rD egdiR timmuS 52821							
notsuoH AIA
...erom daeR
 .yawateg dnekeew dedeen-hcum taht ekat reven dna depparts yllaicnanif era seilimaf ynam ,yletanutrofnU .lufrednow eb nac eltsub dna eltsuh yadyreve eht morf yawa gnitteG
tegduB a no syawateG
weN s'tahW
ytuD artxE - yralaS OEC
yralaS OEC
skniL saxeT
tnemetatS laicnaniF
tropeR tsriF loohcS
secruoseR beW ytefaS
srebmeM draoB
ytinummoC
secruoseR beW rehcaeT
skniL ecnerefeR
tuO/nI kcehC yrotnevnI
ylppA ot woH
noitcudeD noitamrofnI eeyolpmE
nigoL liam-E eeyolpmE
seeyolpmE
draC tropeR laredeF
uneM loohcS
nuF rof tsuJ
kniL beW hgiH .rJ
secruoseR beW tnedutS
draC tropeR loohcS
koobdnaH tnedutS dna tneraP
draC tropeR BLCN
stnedutS
srebmeM draoB
noitartsigeR
koobdnaH tnedutS dna tneraP
ytefaS tenretnI
stneraP rof skniL lufpleH
stneraP
tsiL skniL kc

In [66]:
unique_pages_set = set(all_pages)

In [67]:
len(unique_pages_set)

213

In [68]:
#set already get rid of duplicate strings
ratio_df = pd.DataFrame(np.array(list(unique_pages_set)), columns=["Page"])
ratio_df["Ratios"] = np.nan
ratio_df["Similar"] = np.nan #similar will hold similar indexes, > 0.90 ratios
ratio_df

,Page,Ratios,Similar
0,Aristoi Classical Academy\t \nHome\nEve...,NaN,NaN
1,http://www.aristoiclassical.org/apps/news/cate...,NaN,NaN
2,Aristoi Classical Academy\t \nHome\nEve...,NaN,NaN
3,Aristoi Classical Academy\t \nHome\nEve...,NaN,NaN
4,Aristoi Classical Academy\t \nHome\nEve...,NaN,NaN
5,Aristoi Classical Academy\t \nHome\nEve...,NaN,NaN
6,BEGIN:VCALENDAR\rVERSION:2.0\rPRODID:-//Edlio ...,NaN,NaN
7,Aristoi Classical Academy\t \nHome\nEve...,NaN,NaN
8,Aristoi Classical Academy\t \nHome\nEve...,NaN,NaN
9,Aristoi Classical Academy\t \nHome\nEve...,NaN,NaN


In [69]:
ratio_df['Ratios'] = ratio_df['Ratios'].astype('object')
ratio_df['Similar'] = ratio_df['Similar'].astype('object')
ratio_df.dtypes

Page       object
Ratios     object
Similar    object
dtype: object

In [70]:

index = 0
for page0 in unique_pages_set:
    ratios_list = []
    for page1 in unique_pages_set:
        ratios_list.append(SeqMatcher(None, page0, page1).ratio())
    ratio_df['Ratios'][index] = ratios_list
    sim_ind_list = np.asarray(np.where((np.asarray(ratios_list) > 0.9) & (np.asarray(ratios_list) != 1.0))[0]).tolist()
    ratio_df['Similar'][index] = sim_ind_list
    index+=1

#why does ratio_df change everytime? i see the proportions changing, is the pkled file being updated regularly?


 

In [71]:
ratio_df

,Page,Ratios,Similar
0,Aristoi Classical Academy\t \nHome\nEve...,"[1.0, 0.007516778523489933, 0.3289830727484171...","[5, 7, 9, 11, 13, 14, 15, 17, 19, 20, 22, 25, ..."
1,http://www.aristoiclassical.org/apps/news/cate...,"[0.031140939597315436, 1.0, 0.0068406572979403...",[]
2,Aristoi Classical Academy\t \nHome\nEve...,"[0.3245897402765215, 0.0020819391776340248, 1....",[]
3,Aristoi Classical Academy\t \nHome\nEve...,"[0.6080329058795064, 0.004489337822671156, 0.2...",[]
4,Aristoi Classical Academy\t \nHome\nEve...,"[0.368155566927407, 0.0024034334763948497, 0.2...",[]
5,Aristoi Classical Academy\t \nHome\nEve...,"[0.9949626541601528, 0.0075107296137339056, 0....","[0, 7, 9, 11, 13, 14, 15, 17, 19, 20, 22, 25, ..."
6,BEGIN:VCALENDAR\rVERSION:2.0\rPRODID:-//Edlio ...,"[0.01668534429087287, 0.0036654448517160947, 0...","[24, 91, 100, 102, 117]"
7,Aristoi Classical Academy\t \nHome\nEve...,"[0.9954814042405283, 0.007516778523489933, 0.3...","[0, 5, 9, 11, 13, 14, 15, 17, 19, 20, 22, 25, ..."
8,Aristoi Classical Academy\t \nHome\nEve...,"[0.8128134606050801, 0.00674373795761079, 0.32...",[]
9,Aristoi Classical Academy\t \nHome\nEve...,"[0.9987828203790645, 0.007522837184309511, 0.3...","[0, 5, 7, 11, 13, 14, 15, 17, 19, 20, 22, 25, ..."


In [72]:
len(ratio_df['Ratios'][0])

213

In [15]:
ratio_df['Ratios'][2]

[0.7341719077568134,
 0.7786191536748329,
 1.0,
 0.789272030651341,
 0.8586876382403539,
 0.8903586873569066,
 0.8588293162813576,
 0.8581961169820594,
 0.024616319708251025,
 0.8612887358583374,
 0.9047495458084609,
 0.8676984523215178,
 0.7869438855354348,
 0.8586181460536022,
 0.5167182206510532,
 0.8590405904059041,
 0.8577745025792188,
 0.8895705521472392,
 0.5402741413830279,
 0.7652705061082025,
 0.020854526958290945,
 0.5830039525691699,
 0.8587570621468926,
 0.6209948663480263,
 0.8586181460536022,
 0.618805813342672,
 0.8579852579852579,
 0.8127471505001163,
 0.8586181460536022,
 0.5231136295854458,
 0.8588987217305801,
 0.9040103492884864,
 0.8586181460536022,
 0.7793427230046949,
 0.8579852579852579,
 0.7298538622129437,
 0.6256025709694697,
 0.8584070796460177,
 0.903242542153048,
 0.8995141907440553,
 0.8583353793272772,
 0.9171907756813418,
 0.8027522935779816,
 0.024081313526192336,
 0.7391946025722117,
 0.8891726251276814,
 0.02615174091531093,
 0.8360931125509959,
 0.

In [99]:
#store as (index, cleaned_string) as tuples or lists
#get all index (0, strings)'s and pick shortest one
#add to list of final parse strings > header removed
#reverse strings, and repeat same process so footers removed

In [17]:
list_of_triples = SeqMatcher(lambda x: x == "\n\t", ratio_df['Page'][2], ratio_df['Page'][10]).get_matching_blocks()
list_of_triples

[Match(a=0, b=0, size=1070),
 Match(a=1073, b=1074, size=2),
 Match(a=1090, b=1216, size=5),
 Match(a=1292, b=1229, size=666),
 Match(a=1958, b=1895, size=0)]

In [17]:
ratio_df['Page'][2]

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [18]:
ratio_df['Page'][2] == ratio_df['Page'][0]

False

In [19]:
ratio_df['Page'][4]

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [37]:
i=0
for match in list_of_triples:
    print("overlap " +  str(i) + "\n " + ratio_df['Page'][4][match[1]: match[1]+match[2]])
    i+=1

overlap 0
 Cumberland Academy Middle School
Skip to main content
District
Our Schools
The Leadership Academy
Cumberland Elementary School
Cumberland High School
About Us
Principal's Page
Overview
Staff Directory
Find us on Facebook!
Mission and Vision
What is a Charter School?
CAH Master Plan
Academics
Curriculum
Course Offerings
Saturday School
National Junior Honor Society
Student Organizations and Clubs
Enrollment
2018-2019 Enrollment Information
Enrollment Forms
What is a Charter School?
Frequently Asked Questions
Athletics
Online Shop for Athletic Gear
CAMS Athletics Forms
VB Summer Open Gym Schedule
Schedules
Athletic Director
Our Philosophy
Departments
Required UIL Forms
Cumberland Academy Athletic Boosters
Students
Bell Schedule
Dress Code
Lunch Menu
Bullying Report Form
Extended Day Program
Student Handbook / Code of Conduct
Parents
School Calendar
Parent Portal
Parent Portal Video Tutorial
My School Bucks
Remind Account Information
Texas Minimum Vaccine Requirements
Free and 

In [207]:
m = SeqMatcher(lambda x: x == "\n\t", ratio_df['Page'][0], ratio_df['Page'][14])
m.find_longest_match(0, len(ratio_df['Page'][0]), 0, len(ratio_df['Page'][14]))

Match(a=1236, b=1209, size=666)

In [217]:
ratio_df['Page'][0][1236:1236+666]

'\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Portal'

In [219]:
ratio_df['Page'][14][1209:1209+666]

'\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Portal'

In [220]:
list_of_triples

[Match(a=0, b=0, size=1070),
 Match(a=1147, b=1097, size=10),
 Match(a=1157, b=1123, size=8),
 Match(a=1211, b=1206, size=3),
 Match(a=1236, b=1209, size=666),
 Match(a=1902, b=1875, size=0)]

In [146]:
ratio_df['Page'][0][175:175+10]
#i think take out the first half of overlaps
#if not take out all overlaps -> web pages won't usyally overlap in content, only the header and footerwill most likely overlap

'\nOverview\n'

In [117]:
ratio_df['Page'][2]#[130:130+15]

'">\nCumberland High School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland Middle School\nAbout Us\nAnnouncements\nMeet the Principals\nOverview\nPerforming Arts Center\nFind us on Facebook!\nCAH Master Plan\nTeacher Spotlight\nWhat is a Charter School?\nOur Vision and Mission\nStaff Directory\nAcademics\nCurriculum \nPre-AP / AP Offerings\nDual Credit\nThursday Night in the Classroom\nTJC Promise Scholarship\nACT Dates and Deadlines\nNational Honor Society\nEnrollment\nPreview Night\n2018-2019 Enrollment Information\nEnrollment Forms\nFrequently Asked Questions\nAthletics\nOnline Team Shop for Athletic Gear\nRequired UIL Forms\nAthletic Director\nOur Philosphy\nDepartments\nSchedules\nHS Athletic Handbook\nAthletic Boosters\nStudents\n2017-2018 Yearbook\nRemind\nBell Schedule\nDress Code\nParking Permit\nLunch Menu\nPep Rally Schedule\nStudent Council\nVirtual Library\nBullying Report Form\nStudent Handbook / Code of Cond

In [112]:
k = []
li = []
li.append([2,"hi"])
li.append([4,"zi"])
k.extend(li)
k

[[2, 'hi'], [4, 'zi']]

In [150]:
t = [None]*14
mk= ['Hey', 'Hello' , "what's up"]
count = 0
num = [0,9,10]
for i in mk:
    t.insert(num[count], mk[count])
    count+=1
t

['Hey',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Hello',
 "what's up",
 None,
 None,
 None,
 None,
 None,
 None]

In [73]:
num_rows = ratio_df.shape[0] 
tot_tuples = []

final_cut_strings = [None] * num_rows
count_not_similar = 0  
not_sim_list = []
sim_list = []

for r in range(num_rows):
    if (len(ratio_df['Similar'][r]) != 0):
        sim_list.append(r)
        print("has similar > 0.9 at index : " + str(r))
        list_cut = []
        for ind in ratio_df['Similar'][r]:
            list_of_triples = SeqMatcher(None, ratio_df['Page'][r], ratio_df['Page'][ind]).get_matching_blocks()
            zeroth_triple = list_of_triples[0] #first triple, most likely in beginning, most likely a header or footer
            n = zeroth_triple[2] #j to j+n are the indices of the overlapping part
            orig_string = ratio_df['Page'][ind]
            cut_down_string = orig_string[:zeroth_triple[1]] + orig_string[zeroth_triple[1] + n:] #removes overlapping part
            list_cut.append([ind, cut_down_string])
        tot_tuples.extend(list_cut) #list of tuples
    else:
        print("no similar at index : " + str(r))
        not_sim_list.append(r)
        count_not_similar +=1
        final_cut_strings[r]= ratio_df['Page'][r]

has similar > 0.9 at index : 0
no similar at index : 1
no similar at index : 2
no similar at index : 3
no similar at index : 4
has similar > 0.9 at index : 5
has similar > 0.9 at index : 6
has similar > 0.9 at index : 7
no similar at index : 8
has similar > 0.9 at index : 9
has similar > 0.9 at index : 10
has similar > 0.9 at index : 11
no similar at index : 12
has similar > 0.9 at index : 13
has similar > 0.9 at index : 14
has similar > 0.9 at index : 15
no similar at index : 16
has similar > 0.9 at index : 17
no similar at index : 18
has similar > 0.9 at index : 19
has similar > 0.9 at index : 20
has similar > 0.9 at index : 21
has similar > 0.9 at index : 22
no similar at index : 23
has similar > 0.9 at index : 24
has similar > 0.9 at index : 25
has similar > 0.9 at index : 26
has similar > 0.9 at index : 27
no similar at index : 28
has similar > 0.9 at index : 29
no similar at index : 30
no similar at index : 31
has similar > 0.9 at index : 32
has similar > 0.9 at index : 33
has si

In [19]:
len(final_cut_strings)

164

In [20]:
ratio_df.shape[0] 

164

In [293]:
m = [None]* 5
m[2]= "hi"
m

[None, None, 'hi', None, None]

In [294]:
len(m)

5

In [330]:
len(sim_list) #all indices which shouldare groups, ready to be inserted into final strings

92

In [297]:
final_cut_strings[5] == ratio_df['Page'][5]

True

In [298]:
final_cut_strings[0] == ratio_df['Page'][0] 

False

In [299]:
final_cut_strings[35] == ratio_df['Page'][35]

True

In [256]:
print(final_cut_strings[0])

None


In [24]:
ratio_df['Page'][35]

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [21]:
tot_tuples

[[10,
  "Bama Godeaux\nName\n\t\t\t\t\t\t\t Bama  Godeaux\t\t\t\t\t\t\nPosition\nInclusion\nEmail\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSend email\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\nLess info\n\t\t\t\t\tSubscribe\t\t\t\t\nWhat's this?\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Portal"],
 [31,
  "About\n\t\t\t » What is a Charter School?\t\t\t\nWhat is a Ch

In [23]:
count = 0
for i in final_cut_strings:
    if i is  None:
        print("indices with similar that have multiple: " + str(count))
    count+=1

indices with similar that have multiple: 2
indices with similar that have multiple: 4
indices with similar that have multiple: 5
indices with similar that have multiple: 6
indices with similar that have multiple: 7
indices with similar that have multiple: 8
indices with similar that have multiple: 9
indices with similar that have multiple: 10
indices with similar that have multiple: 13
indices with similar that have multiple: 15
indices with similar that have multiple: 16
indices with similar that have multiple: 17
indices with similar that have multiple: 20
indices with similar that have multiple: 22
indices with similar that have multiple: 24
indices with similar that have multiple: 26
indices with similar that have multiple: 28
indices with similar that have multiple: 30
indices with similar that have multiple: 31
indices with similar that have multiple: 32
indices with similar that have multiple: 34
indices with similar that have multiple: 37
indices with similar that have multiple

In [212]:
j = [[] for x in range(4)] 
j[2].append("hi")
j[2].append("hey")
j[1].append("duh dah duh dah")
j

[[], ['duh dah duh dah'], ['hi', 'hey'], []]

In [74]:
list_grouped = [[] for x in range(num_rows)] #big list, just put list of strings in spots where needed
list_indices_of_groups = []
for ind in range(num_rows):
    for tup in tot_tuples:
        if(tup[0] == ind):
            list_grouped[ind].append(tup[1]) #attach that tuple's string
            list_indices_of_groups.append(tup[0])

In [75]:
unique_group_ind = set(list_indices_of_groups)

In [76]:
ind_fill_final_grouped = []
i = 0
for group in list_grouped:
    if (len(group) != 0):
        #print("list ready to insert at index " + str(i) + "\n")
        ind_fill_final_grouped.append(i)
    
    i+=1
        

In [77]:
spot = 0
for li in list_grouped:
    if (len(li) != 0):
        #print(unique_ind_list[spot])
        final_cut_strings[ind_fill_final_grouped[spot]]= min(li, key=len) #inserts into correct index, what was None before, add in that string now
        spot+=1
        
#was able to parse corresponding strings properly to cut down version

In [78]:
len(list_grouped)

213

In [79]:
len(final_cut_strings)

213

In [30]:
index = 0
for s in final_cut_strings:
    if (s not in ratio_df['Page'][index]):
        print("not contains in index: " + str(index))
    index+=1

In [31]:
ratio_df['Page'][0] == final_cut_strings[0]

True

In [35]:
final_cut_strings[10] in ratio_df['Page'][10] 

True

In [37]:
final_cut_strings[10]

"Bama Godeaux\nName\n\t\t\t\t\t\t\t Bama  Godeaux\t\t\t\t\t\t\nPosition\nInclusion\nEmail\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSend email\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\nLess info\n\t\t\t\t\tSubscribe\t\t\t\t\nWhat's this?\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Portal"

In [38]:
ratio_df['Page'][10] 

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [358]:
# t = [None]*12
# mk= ['Hey', 'Hello' , "what's up"]
# count = 0
# num = [0,9,10]
# for i in mk:
#     t.insert(num[count], mk[count])
#     count+=1
#for m in t:
    #if (m is None):
        #print(m)

In [367]:
# r = [None] * 5
# first = ["hiheosfah", 'smallest', "not smallest"]
# second = ["fdsjlkajflkdfslkaf", 'second', "fjdsklajflkdsajflksjflkjdlasfd"]
# longli = []
# longli.append([])
# longli.append(first)
# longli.append([])
# longli.append(second)
# count = 0
# for li in longli:
#     if (len(li) != 0):
#         r.insert(count, min(li, key=len))
#     count+=1
# #r

In [50]:
final_cut_strings[0]

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [51]:
ratio_df['Page'][0]


"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [39]:
for s in final_cut_strings:
    print(s.find('.'))

1134
1339
483
1073
96
497
96
97
6
96
420
1171
1110
96
1133
94
99
463
1153
1122
154
1310
100
1133
96
1250
99
1072
97
1133
97
432
95
1193
99
1089
1140
97
421
108
102
68
1220
-1
1138
483
5
126
94
424
1340
1284
92
96
95
1132
1198
55
1194
95
97
37
96
1287
96
1250
513
1255
130
-1
93
95
96
95
1228
17
144
1530
148
1171
156
491
1222
423
99
1258
1073
94
436
92
446
195
18
58
23
61
1777
1287
97
421
98
1275
1276
215
1094
98
1253
168
83
93
5
96
1133
1206
96
54
1282
96
9
1133
1132
1073
98
1133
92
1212
1106
1137
54
1129
95
1403
1089
399
96
48
1272
1281
1300
98
1073
1140
1073
94
99
98
86
1224
1135
98
1357
48
93
521
1323
96
432
433
83
48
96
1205
100
102


In [80]:
fin_ind_sum = 0
for ind in range(num_rows):
    punc_ind = final_cut_strings[ind].find(",")
    if(punc_ind == -1):
        punc_ind = 0

    is_cut_down = ind in unique_group_ind
    fin_ind_sum += punc_ind
    print("At index : " + str(ind) + " . punc ind is : " + str(punc_ind) + ". Is a cut down string originally? : " + str(is_cut_down))

At index : 0 . punc ind is : 48. Is a cut down string originally? : True
At index : 1 . punc ind is : 220. Is a cut down string originally? : False
At index : 2 . punc ind is : 232. Is a cut down string originally? : False
At index : 3 . punc ind is : 232. Is a cut down string originally? : False
At index : 4 . punc ind is : 232. Is a cut down string originally? : False
At index : 5 . punc ind is : 54. Is a cut down string originally? : True
At index : 6 . punc ind is : 0. Is a cut down string originally? : True
At index : 7 . punc ind is : 50. Is a cut down string originally? : True
At index : 8 . punc ind is : 232. Is a cut down string originally? : False
At index : 9 . punc ind is : 48. Is a cut down string originally? : True
At index : 10 . punc ind is : 6. Is a cut down string originally? : True
At index : 11 . punc ind is : 49. Is a cut down string originally? : True
At index : 12 . punc ind is : 232. Is a cut down string originally? : False
At index : 13 . punc ind is : 3. Is a 

In [60]:
final_cut_strings[59]

'6th and 7th Grade Awards\nWhat\n\t\t\t\t6th and 7th Grade Awards\t\t\t\nWhen\n                5/30/2018            \nCopy to Google Calendar\n\t\xa0•\xa0\t\nDownload iCal Event\nEvents\n← Back to events list\n            \xa0•\xa0            \nOpen calendar grid\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Portal'

In [61]:
ratio_df['Page'][59]

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [65]:
final_cut_strings[114]

"Vikki Sanders\n\t\t\t » Home\t\t\t\nHome\nName\n\t\t\t\t\t\t\t Vikki  Sanders\t\t\t\t\t\t\nPosition\nSpecial Education\nEmail\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSend email\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\nLess info\n\t\t\t\t\tSubscribe\t\t\t\t\nWhat's this?\n\t\t\t\t\t\t\t\t\tHome\t\t\t\t\t\t\t\t\nPhoto Album\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParen

In [66]:
ratio_df['Page'][114]

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [71]:
n_list = [index for index, k in enumerate("hello how \n are you, \nwassss uuppp?") if k=='\n']
for i in n_list:
    print(i)
        

10
21


In [41]:
start = 0
end = 0
total = ""
list_totals = []
st_en = []
s = ratio_df['Page'][114]
for c in s:
    if (c not in ['\n', '\t']):
        total+=(c)
        end+=1
    else :
        if(len(total.split()) >= 7): # we hit 7 words or more, wipe everything before start index
            list_totals.append(total)
            st_en.append((start, end))
    
        total= ""
        start = end

list_totals
st_en
#cut till start index of first tuple, tuple[0][0]


[(1122, 1230),
 (1230, 1301),
 (1338, 1366),
 (1460, 1623),
 (1623, 1841),
 (1862, 1936)]

In [81]:
super_final_strings = []
for s in final_cut_strings:
    use_punc = False
    use_sev = False
    punc = [",", ".", ":", ";"]
    p_list = []
    for p in punc:
        if (s.find(p) != -1):
            p_list.append(s.find(p))
        else:
            p_list.append(len(s))
    
    punc_ind = min(p_list)
    
    n_list = [index for index, k in enumerate(s) if k=='\n']
    start_punc = len(s)
    for i in n_list:
        if(i < punc_ind):
            start_punc = i # r equals the largest index of \n that's less than index of first punctuation
    
    start = 0
    end = 0
    total = ""
    list_totals = []
    st_en = []
    for c in s:
        if (c not in ['\n', '\t']):
            total+=(c)
            end+=1
        else :
            if(len(total.split()) >= 7): # we hit 7 words or more, wipe everything before start index
                #list_totals.append(total)
                st_en.append((start, end))
    
            total= ""
            start = end
    start_sev = len(s)-1
    if len(st_en) > 0:
        start_sev = st_en[0][0] #index of first group of words that's >= 7 words; 0th tuple's start value
    #else :
        #start_sev = len(s)-1 #make it huge if there's no group of 7, so the start punc has to be the smalles
            
    
    
    #take smaller of the two indices, since we want to use the property which occurs first
    if start_punc < start_sev:
        #if start of sentence which ends in/contains puncuation occurs earlier, wipe eveything before that index
        #only take that index +1 and on, start right after the new line
        new_string = s[start_punc+1:] 
        super_final_strings.append(new_string)
        
    else:
        #if start of group of words that >= 7 occurs earlier than a sentence with punctuation, wipe eveything before that index
        #only take that index and on, statr using that begining of the group of 7+ words
        new_string = s[start_sev:] 
        super_final_strings.append(new_string)
        
     
            


In [82]:
super_final_strings # cleaned

['Sorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\n©Aristoi Classical Academy\n5618 Eleventh St.\nKaty,\xa0 TX\xa0 77493\nPhone:\xa0 281-391-5003\nFax: 281-391-5010\nPowered by Edlio',
 '0',
 '    \t\t\nSearch\nSearch\n\t\t\t"Aristoi" is an ancient Greek word meaning "the best, noble character, and a right nature."\t\t\nCultivating Wisdom and Virtue\nTuition-Free\nABOUT US\nAt a Glance\nMission, Vision, Philosophy\nSchool Profile Narrative\nCore Beliefs\nThe Virtues\nWhat is Classical Education?\nWhat is a Charter School?\nLove & Logic\nOur Crest\nOur Mascot\nA Word from Administration\nHeadmaster/ Superintendent\nAssistant Headmaster/ Dean of Students\nAssistant Headmaster/ Dean of Academics\nAssistant Headmas

In [83]:
superfin_ind_sum = 0
for ind in range(num_rows):
    punc_ind = super_final_strings[ind].find(",")
    if (punc_ind == -1):
        punc_ind = 0
    is_cut_down = ind in unique_group_ind
    superfin_ind_sum+=punc_ind
    print("At index : " + str(ind) + " . punc ind is : " + str(punc_ind) + ". Is a cut down string originally? : " + str(is_cut_down))
        
#yay! was able to cut down the strings quite a bit!! :)

At index : 0 . punc ind is : 5. Is a cut down string originally? : True
At index : 1 . punc ind is : 0. Is a cut down string originally? : False
At index : 2 . punc ind is : 76. Is a cut down string originally? : False
At index : 3 . punc ind is : 76. Is a cut down string originally? : False
At index : 4 . punc ind is : 76. Is a cut down string originally? : False
At index : 5 . punc ind is : 5. Is a cut down string originally? : True
At index : 6 . punc ind is : 0. Is a cut down string originally? : True
At index : 7 . punc ind is : 5. Is a cut down string originally? : True
At index : 8 . punc ind is : 76. Is a cut down string originally? : False
At index : 9 . punc ind is : 5. Is a cut down string originally? : True
At index : 10 . punc ind is : 0. Is a cut down string originally? : True
At index : 11 . punc ind is : 59. Is a cut down string originally? : True
At index : 12 . punc ind is : 76. Is a cut down string originally? : False
At index : 13 . punc ind is : 0. Is a cut down st

In [84]:
print("fin ind sum : " + str(fin_ind_sum) + " . " + "super fin ind sum : " + str(superfin_ind_sum))

fin ind sum : 28602 . super fin ind sum : 11577


In [85]:
final_cut_strings [147]

'ndrick Davis\n\t\t\tPlease verify your identity\t\t\nSorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\n©Aristoi Classical Academy\n5618 Eleventh St.\nKaty,\xa0 TX\xa0 77493\nPhone:\xa0 281-391-5003\nFax: 281-391-5010\nPowered by Edlio'

In [86]:
super_final_strings[147]

'Sorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\n©Aristoi Classical Academy\n5618 Eleventh St.\nKaty,\xa0 TX\xa0 77493\nPhone:\xa0 281-391-5003\nFax: 281-391-5010\nPowered by Edlio'

In [47]:
len(final_cut_strings [147]) - len(super_final_strings[147])

1081

In [48]:
footers = []
for s in super_final_strings:
    per_ind_li = [ind for ind, char in enumerate(s) if char == "."]
    per_ind = 0
    if len(per_ind_li) >=2 :
        per_ind = per_ind_li[-2]

    #get 2nd to last one since very last period ends after Copyright © Cumberland Academy Middle School.
    # > which is part of the "footer" I suppose
    footers.append(len(s) - per_ind)
    
footers

[187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 1,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 92,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 1,
 187,
 187,
 1,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 3,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 92,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 39,
 187,
 187,
 187,
 187,
 187,
 187,
 1,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 1,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 128,
 187,
 187,
 187,
 187,
 187,
 187,
 92,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 92,
 187,
 187,
 187,
 187,
 187,
 187,
 187,
 92,
 187,
 187,
 187,
 187]

In [141]:
len(" \xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Portal")

#this is the same footer used over and over again, this is just for this school

187

In [ ]:
#now remove the text after the second to last period? aka remove text after the last relevant sentence 
#most footers that are there are : 
#\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Portal

In [191]:
s

'n Grote\n\t\t\tPlease verify your identity\t\t\nSorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy

In [49]:
footer_list = ["Twitter", "Copyright", "All Rights Reserved",  "Read More", 
               "Useful Links", "Search", "Survey", "Feed", "Phone", "Fax", "Address",  "Sitemap", 
              "Jobs", "Apply", "Pre-Enroll"]
s = "n Grote\n\t\t\tPlease verify your identity\t\t\nSorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Porta"
no_newline = s.replace("\n", " ")
new_list = []
for word in footer_list:
    if (no_newline.find(word) != -1):
        new_list.append(no_newline.find(word))
    else:
        new_list.append(len(s))

        
f_ind = min(new_list)
footer_rem = s[:f_ind]
footer_rem 


'n Grote\n\t\t\tPlease verify your identity\t\t\nSorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\n'

In [50]:
n_list = [index for index, k in enumerate(s) if k in ['\n', '.']]
n_list

[7,
 40,
 75,
 92,
 93,
 201,
 202,
 273,
 274,
 285,
 297,
 306,
 315,
 344,
 354,
 361,
 372,
 380,
 386,
 397,
 406,
 426,
 452,
 615,
 616,
 833,
 835,
 844,
 852,
 859,
 904,
 934,
 947,
 952,
 978,
 1006]

In [87]:
footer_list = ["Copyright", "All Rights Reserved",  "Read More", 
               "Useful Links", "Search", "Survey", "Feed", "Phone", "Fax", "Address",  "Sitemap", 
               "Careers", "Jobs", "Apply", "Pre-Enroll"]

#facebook, contact us, enroll etc occurs in headers as well, so had to take that out
footers_removed_strings = []

for s in super_final_strings:
    no_newline = s.replace("\n", " ")
    new_list = []
    for word in footer_list:
        if (no_newline.find(word) != -1):
            new_list.append(no_newline.find(word))
        else :
            new_list.append(len(s))

    f_ind = min(new_list)
    #go back to new line period right before and wipe out everything after that
    n_list = [index for index, k in enumerate(s) if k in ['\n', '.']]
    start_punc = len(s)
    for i in n_list:
        if(i < f_ind):
            start_punc = i # r equals the largest index of \n that's less than index of the keyword
    if start_punc < f_ind:
              
            footer_rem = s[:start_punc]
            footers_removed_strings.append(footer_rem) 
    else:
            footer_rem = s[:f_ind]
            footers_removed_strings.append(footer_rem) 
              

In [88]:
footers_removed_strings

['Sorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\n©Aristoi Classical Academy\n5618 Eleventh St.\nKaty,\xa0 TX\xa0 77493',
 '0',
 '    \t\t',
 '    \t\t',
 '    \t\t',
 'Sorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\n©Aristoi Classical Academy\n5618 Eleventh St.\nKaty,\xa0 TX\xa0 77493',
 '\r',
 'Sorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by ch

In [54]:

for count in range(len(footers_removed_strings)):
    print("Diff at index : " + str(count) + " . length in cut before footers - after : " + str(len(super_final_strings[count])- len(footers_removed_strings[count])))

Diff at index : 0 . length in cut before footers - after : 168
Diff at index : 1 . length in cut before footers - after : 168
Diff at index : 2 . length in cut before footers - after : 168
Diff at index : 3 . length in cut before footers - after : 168
Diff at index : 4 . length in cut before footers - after : 168
Diff at index : 5 . length in cut before footers - after : 168
Diff at index : 6 . length in cut before footers - after : 168
Diff at index : 7 . length in cut before footers - after : 168
Diff at index : 8 . length in cut before footers - after : 0
Diff at index : 9 . length in cut before footers - after : 168
Diff at index : 10 . length in cut before footers - after : 168
Diff at index : 11 . length in cut before footers - after : 168
Diff at index : 12 . length in cut before footers - after : 168
Diff at index : 13 . length in cut before footers - after : 168
Diff at index : 14 . length in cut before footers - after : 1341
Diff at index : 15 . length in cut before footers -

In [55]:
super_final_strings[10]

'nts\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMAC\nMy School Account - Lunch\nOverview of New High School\nParent Portal'

In [56]:
final_cut_strings[7]

'nnifer Davis\n\t\t\tPlease verify your identity\t\t\nSorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter\nSearch\nCopyright © Cumberland Academy Middle School. Contact us at: 903-504-5393 \nUseful Links\nDMA

In [57]:
footers_removed_strings[7]

'Sorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter'

In [61]:
ratio_df['Page'][7]

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [62]:
ratio_df['Page'][98]

"Cumberland Academy Middle School\nSkip to main content\nDistrict\nOur Schools\nThe Leadership Academy\nCumberland Elementary School\nCumberland High School\nAbout Us\nPrincipal's Page\nOverview\nStaff Directory\nFind us on Facebook!\nMission and Vision\nWhat is a Charter School?\nCAH Master Plan\nAcademics\nCurriculum\nCourse Offerings\nSaturday School\nNational Junior Honor Society\nStudent Organizations and Clubs\nEnrollment\n2018-2019 Enrollment Information\nEnrollment Forms\nWhat is a Charter School?\nFrequently Asked Questions\nAthletics\nOnline Shop for Athletic Gear\nCAMS Athletics Forms\nVB Summer Open Gym Schedule\nSchedules\nAthletic Director\nOur Philosophy\nDepartments\nRequired UIL Forms\nCumberland Academy Athletic Boosters\nStudents\nBell Schedule\nDress Code\nLunch Menu\nBullying Report Form\nExtended Day Program\nStudent Handbook / Code of Conduct\nParents\nSchool Calendar\nParent Portal\nParent Portal Video Tutorial\nMy School Bucks\nRemind Account Information\nTexas

In [78]:
footers_removed_strings[98]

'Sorry, your authentication failed!\nPlease try again.\nWe are only accepting secure messages at this time, please verify your identity by choosing an option below.\nYou will not be able to fill out the form until you complete this step.\nYour Name*\nYour Email*\nSubject*\nMessage*\nSend me a copy of this email\n\t\t\tSend\t\t\nEnroll\nEmployment\nParents\nStaff\nContact Us\nFacebook\nCumberland Facebook\nCumberland Academy Middle\nAt Cumberland Academy Middle School,\xa0we promote the characteristics of a successful student: respect, responsibility, trust, caring, perseverance, and citizenship.\nOur vision is to empower student success by providing a nurturing environment that inspires lifelong learning while using innovative technology to develop 21st century skills and fostering an appreciation of the arts.\xa0\nFacebook\nTwitter'

In [89]:
k = list(unique_pages_set)
for i in range(len(k)):
    if "Estrella Mountain" in footers_removed_strings[i]:
        print(str(i))

20
135
151
159


In [131]:
footers_removed_strings[205]

'BeMent, Felicity\n »            \nMain »    \n    There are no photos in our album at this point.  Please check back soon...\n©Aristoi Classical Academy\n5618 Eleventh St.\nKaty,\xa0 TX\xa0 77493'

In [132]:
ratio_df['Page'][205]

'Aristoi Classical Academy\t        \nHome\nEvents\n2017-2018 Academic Calendar\n2018-2019 Academic Calendar\nContact Us\nStaff Login\nSearch    \t\t    \t\t    \t\t    \t\t    \t\t\nSearch\nSearch\n\t\t\t"Aristoi" is an ancient Greek word meaning "the best, noble character, and a right nature."\t\t\nCultivating Wisdom and Virtue\nTuition-Free\nABOUT US\nAt a Glance\nMission, Vision, Philosophy\nSchool Profile Narrative\nCore Beliefs\nThe Virtues\nWhat is Classical Education?\nWhat is a Charter School?\nLove & Logic\nOur Crest\nOur Mascot\nA Word from Administration\nHeadmaster/ Superintendent\nAssistant Headmaster/ Dean of Students\nAssistant Headmaster/ Dean of Academics\nAssistant Headmaster, Elementary School\nParent Communications\nGiving\nDonations & Fundraising\nDonate Online\nBoxtops\n5th Griffin Gala\nBoard of Directors\nBoard Meetings\nBoard Members\nAdministration, Faculty, & Staff\nLocation\nEmployment\nAristoi News\nPublications\nENROLLMENT\nEnrollment Application\nRegistr

In [123]:
for i in range(len(footers_removed_strings)):
    if (len(footers_removed_strings[i]) > 15 and ("Email" not in footers_removed_strings[i])):
        print(i)

38
48
58
87
99
126
130
137
144
165
169
181
188
205
